**`This Notebook Done By :`**
- **Salma Shaheen `220210654`**

- **Rasha Abu Rkab `220210069`**
---

### `Web Scraping - Al Jazeera Channel`

#### Install the main libraries

In [1]:
'''pip install lxml_html_clean'''

'pip install lxml_html_clean'

In [2]:
'''pip install requests beautifulsoup4 pandas tqdm dateparser'''

'pip install requests beautifulsoup4 pandas tqdm dateparser'

In [144]:
'''!pip install newspaper3k'''

'!pip install newspaper3k'

In [3]:
'''pip install webdriver-manager'''

'pip install webdriver-manager'

In [4]:
'''!pip install selenium
!wget -q https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.68/linux64/chrome-linux64.zip
!unzip -q chrome-linux64.zip
!wget -q https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.68/linux64/chromedriver-linux64.zip
!unzip -q chromedriver-linux64.zip'''

'!pip install selenium\n!wget -q https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.68/linux64/chrome-linux64.zip\n!unzip -q chrome-linux64.zip\n!wget -q https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.68/linux64/chromedriver-linux64.zip\n!unzip -q chromedriver-linux64.zip'

#### Import the main Libraries

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import urljoin, quote
import time
import pandas as pd
from google.colab import files
import os
import re
from newspaper import Article
from urllib.parse import urlparse

#### Perform the web scraping from `https://www.aljazeera.net` :
- Using the `Searching` the platform
- Then enter the date like : `January 01 2024`
- Using one of the Specific keywords like :`["فلسطين","غزة"]`
- Do a loop to perform that for a `31` days in January months to each  keyword 

In [ ]:
# Check paths
assert os.path.exists("/content/chrome-linux64/chrome"), "Chrome not found!"
assert os.path.exists("/content/chromedriver-linux64/chromedriver"), "Chromedriver not found!"

# Configure browser
options = Options()
options.binary_location = "/content/chrome-linux64/chrome"
options.add_argument("--headless=new")  # for Chrome 109+
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

service = Service("/content/chromedriver-linux64/chromedriver")

# Constants
BASE = "https://www.aljazeera.net"
keywords = ["فلسطين","غزة"]
year = "2024"
all_urls = set()

# Loop through keywords and dates
for keyword in keywords:
    keyword_encoded = quote(keyword)
    for day in range(1, 32):
        date_str = f"January {day} {year}"
        date_encoded = quote(date_str)
        search_url = f"https://www.aljazeera.net/search/{date_encoded}%20{keyword_encoded}"
        print(f"\n🔍 Searching: {search_url}")

        driver = webdriver.Chrome(service=service, options=options)
        driver.get(search_url)

        # Close cookies popup
        try:
            cookie_button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
            cookie_button.click()
            time.sleep(2)
        except:
            pass

        # Load more results
        for _ in range(40):
            try:
                show_more = driver.find_element(By.CLASS_NAME, "show-more-button")
                driver.execute_script("arguments[0].scrollIntoView(true);", show_more)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", show_more)
                time.sleep(3)
            except Exception:
                break

        # Extract links
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        for a in soup.find_all("a", href=True):
            link = urljoin(BASE, a["href"])
            if link.startswith(BASE + "/news/") and f"/{year}/" in link:
                #if "liveblog" not in link and "gallery" not in link:
                all_urls.add(link)

        driver.quit()

# Output results
print(f"\n✅ Found {len(all_urls)} news links from January {year}:")
for url in sorted(all_urls):
    print(url)



🔍 Searching: https://www.aljazeera.net/search/January%201%202024%20%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86

🔍 Searching: https://www.aljazeera.net/search/January%202%202024%20%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86

🔍 Searching: https://www.aljazeera.net/search/January%203%202024%20%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86

🔍 Searching: https://www.aljazeera.net/search/January%204%202024%20%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86

🔍 Searching: https://www.aljazeera.net/search/January%205%202024%20%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86

🔍 Searching: https://www.aljazeera.net/search/January%206%202024%20%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86

🔍 Searching: https://www.aljazeera.net/search/January%207%202024%20%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86

🔍 Searching: https://www.aljazeera.net/search/January%208%202024%20%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86

🔍 Searching: https://www.aljazeera.net/search/January%209%202024%20%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86

🔍 Searching: https://www.aljazeera.net/search/January%

In [ ]:
df = pd.DataFrame(list(all_urls), columns=['URL'])
df_Jazera_Jan=pd.read_csv("/content/Aljazeera_January_News_final.csv")
df_Jazera_Jan

,URL
0,https://www.aljazeera.net/news/2024/1/1/%D8%B2...
1,https://www.aljazeera.net/news/2024/1/1/%D8%B9...
2,https://www.aljazeera.net/news/2024/1/1/%D9%83...
3,https://www.aljazeera.net/news/2024/1/1/%D9%85...
4,https://www.aljazeera.net/news/2024/1/1/%D9%85...
...,...
793,https://www.aljazeera.net/news/liveblog/2024/9...
794,https://www.aljazeera.net/news/longform/2024/1...
795,https://www.aljazeera.net/news/longform/2024/1...
796,https://www.aljazeera.net/news/longform/2024/1...


***
***

### `Extract Data (Title + Contant) from URL - Al Jazeera`

In [ ]:
titles = []
contents = []

for url in df_Jazera_Jan['URL']:
    try:
        article = Article(url, language='ar')
        article.download()
        article.parse()

        titles.append(article.title) #Title 
        contents.append(article.text) #Contant

        time.sleep(1)

df_Jazera_Jan["Title"] = titles
df_Jazera_Jan["Content"] = contents
df_Jazera_Jan

,URL,Title,Content
0,https://www.aljazeera.net/news/2024/1/1/%D8%B2...,زلزال بقوة 7.4 درجات يضرب اليابان ويتسبب بموجا...,قالت هيئة الإذاعة والتلفزيون اليابانية إن زلزا...
1,https://www.aljazeera.net/news/2024/1/1/%D8%B9...,عشية رأس السنة.. مظاهرات داعمة لغزة حول العالم,احتفل العالم بالسنة الميلادية الجديدة 2024، ول...
2,https://www.aljazeera.net/news/2024/1/1/%D9%83...,"كيف تفاعل جمهور المنصات مع قصة ""الجندي الإسرائ...",لم يستغرب جمهور وسائل التواصل الاجتماعي من الف...
3,https://www.aljazeera.net/news/2024/1/1/%D9%85...,ما وراء سحب واشنطن حاملة طائرات رغم استمرار حر...,أجمع خبراء على أن قرار واشنطن سحب حاملة الطائر...
4,https://www.aljazeera.net/news/2024/1/1/%D9%85...,مسؤول أميركي: سحب إسرائيل بعض قواتها من غزة بد...,قال مسؤول أميركي اليوم الاثنين إن قرار إسرائيل...
...,...,...,...
793,https://www.aljazeera.net/news/liveblog/2024/9...,الحرب على غزة مباشر.. الإعلام الإسرائيلي يتحدث...,في اليوم الـ357 للحرب على غزة، جدد الاحتلال ال...
794,https://www.aljazeera.net/news/longform/2024/1...,محكمة لاهاي وحرب غزة.. مراحل التقاضي وثماره ال...,شهدت محكمة العدل الدولية في لاهاي يومي 11 و12 ...
795,https://www.aljazeera.net/news/longform/2024/1...,ألمانيا تتجاوز نظيراتها.. أتاحت للاجئيها السور...,لم يكن يوم الثلاثاء 30 سبتمبر/أيلول 2014 يشبه ...
796,https://www.aljazeera.net/news/longform/2024/1...,"""حصانة راندال"" ليست بمتناولهم..صحفيو فلسطين يد...",وذكّر أبو بكر المدعي العام بشكويين سابقتين قدم...


### `Extract Date from the URL link`

In [ ]:
def extract_date(url):
    path = urlparse(url).path
    parts = path.strip("/").split("/")
    if len(parts) >= 4 and parts[1].isdigit():
        year, month, day = parts[1], parts[2], parts[3]
        return f"{year}-{month.zfill(2)}-{day.zfill(2)}"
    elif len(parts) >= 5 and parts[1] in ['longform', 'liveblog']:
        year, month, day = parts[2], parts[3], parts[4]
        return f"{year}-{month.zfill(2)}-{day.zfill(2)}"
    return None

df_ALJazeera_Jan = ALJazeera_Jan.copy()
df_ALJazeera_Jan["date"] = df_ALJazeera_Jan["URL"].apply(extract_date)
df_ALJazeera_Jan["date"] = pd.to_datetime(df_ALJazeera_Jan["date"], errors="coerce")
df_ALJazeera_Jan = df_ALJazeera_Jan[df_ALJazeera_Jan["date"].dt.month == 1]

df_ALJazeera_Jan

,URL,Title,Content,date
0,https://www.aljazeera.net/news/2024/1/1/%D8%B2...,زلزال بقوة 7.4 درجات يضرب اليابان ويتسبب بموجا...,قالت هيئة الإذاعة والتلفزيون اليابانية إن زلزا...,2024-01-01
1,https://www.aljazeera.net/news/2024/1/1/%D8%B9...,عشية رأس السنة.. مظاهرات داعمة لغزة حول العالم,احتفل العالم بالسنة الميلادية الجديدة 2024، ول...,2024-01-01
2,https://www.aljazeera.net/news/2024/1/1/%D9%83...,"كيف تفاعل جمهور المنصات مع قصة ""الجندي الإسرائ...",لم يستغرب جمهور وسائل التواصل الاجتماعي من الف...,2024-01-01
3,https://www.aljazeera.net/news/2024/1/1/%D9%85...,ما وراء سحب واشنطن حاملة طائرات رغم استمرار حر...,أجمع خبراء على أن قرار واشنطن سحب حاملة الطائر...,2024-01-01
4,https://www.aljazeera.net/news/2024/1/1/%D9%85...,مسؤول أميركي: سحب إسرائيل بعض قواتها من غزة بد...,قال مسؤول أميركي اليوم الاثنين إن قرار إسرائيل...,2024-01-01
...,...,...,...,...
768,https://www.aljazeera.net/news/liveblog/2024/1...,"الحرب على غزة مباشر.. ""أقسى يوم"" على جيش الاحت...",في اليوم الـ94 من العدوان على غزة، تكبّدت قوات...,2024-01-08
769,https://www.aljazeera.net/news/liveblog/2024/1...,الحرب على غزة مباشر.. الاحتلال يتكبد خسائر فاد...,في اليوم الـ95 للعدوان على غزة، أعلن الجيش الإ...,2024-01-09
794,https://www.aljazeera.net/news/longform/2024/1...,محكمة لاهاي وحرب غزة.. مراحل التقاضي وثماره ال...,شهدت محكمة العدل الدولية في لاهاي يومي 11 و12 ...,2024-01-11
795,https://www.aljazeera.net/news/longform/2024/1...,ألمانيا تتجاوز نظيراتها.. أتاحت للاجئيها السور...,لم يكن يوم الثلاثاء 30 سبتمبر/أيلول 2014 يشبه ...,2024-01-17


In [268]:
#Organize the Al Jazeera Dataset into Dataframe

AlJazeera={}
AlJazeera["Date"] = df_ALJazeera_Jan['date']
AlJazeera["Platform"] = "Aljazeera"
AlJazeera["Title"] = df_ALJazeera_Jan['title']
AlJazeera["News Content"] = df_ALJazeera_Jan['News content']
AlJazeera["Label"] = "Real"
AlJazeera=pd.DataFrame(AlJazeera)
AlJazeera

,Date,Platform,Title,News Content,Label
0,2024-01-01,Aljazeera,زلزال بقوة 7.4 درجات يضرب اليابان ويتسبب بموجا...,قالت هيئة الإذاعة والتلفزيون اليابانية إن زلزا...,Real
1,2024-01-01,Aljazeera,عشية رأس السنة.. مظاهرات داعمة لغزة حول العالم,احتفل العالم بالسنة الميلادية الجديدة 2024، ول...,Real
2,2024-01-01,Aljazeera,"كيف تفاعل جمهور المنصات مع قصة ""الجندي الإسرائ...",لم يستغرب جمهور وسائل التواصل الاجتماعي من الف...,Real
3,2024-01-01,Aljazeera,ما وراء سحب واشنطن حاملة طائرات رغم استمرار حر...,أجمع خبراء على أن قرار واشنطن سحب حاملة الطائر...,Real
4,2024-01-01,Aljazeera,مسؤول أميركي: سحب إسرائيل بعض قواتها من غزة بد...,قال مسؤول أميركي اليوم الاثنين إن قرار إسرائيل...,Real
...,...,...,...,...,...
243,2024-01-08,Aljazeera,"الحرب على غزة مباشر.. ""أقسى يوم"" على جيش الاحت...",في اليوم الـ94 من العدوان على غزة، تكبّدت قوات...,Real
244,2024-01-09,Aljazeera,الحرب على غزة مباشر.. الاحتلال يتكبد خسائر فاد...,في اليوم الـ95 للعدوان على غزة، أعلن الجيش الإ...,Real
245,2024-01-11,Aljazeera,محكمة لاهاي وحرب غزة.. مراحل التقاضي وثماره ال...,شهدت محكمة العدل الدولية في لاهاي يومي 11 و12 ...,Real
246,2024-01-17,Aljazeera,ألمانيا تتجاوز نظيراتها.. أتاحت للاجئيها السور...,لم يكن يوم الثلاثاء 30 سبتمبر/أيلول 2014 يشبه ...,Real


---
---
## `Misbar Data - Content + Title Extraction`

In [145]:
tel_data = pd.read_csv(r"C:\Users\salma\OneDrive\سطح المكتب\NLP3\telegram_news_2024.csv")
tel_data.head()

,date,channel,text
0,2024-01-01 00:08:44,AJPalestine,ارتفاع عدد الصحفيين الشهداء في قطاع غزة منذ 7 ...
1,2024-01-01 00:12:40,AJPalestine,ارتفاع حصيلة ضحايا الحرب الإسرائيلية على القطا...
2,2024-01-01 00:16:58,AJPalestine,بمشاركة الفنانة الأردنية جولييت عواد.. مسيرة ت...
3,2024-01-01 00:24:58,AJPalestine,عاجل | مراسل #الجزيرة: غارات إسرائيلية على منا...
4,2024-01-01 00:26:11,AJPalestine,من وسط الحرب.. صحفيو غزة يعبرون عن أمنياتهم مع...


In [146]:
# January Data Filter
jan_data = tel_data[tel_data['date'] <= '2024-01-31']
jan_data.head()

,date,channel,text
0,2024-01-01 00:08:44,AJPalestine,ارتفاع عدد الصحفيين الشهداء في قطاع غزة منذ 7 ...
1,2024-01-01 00:12:40,AJPalestine,ارتفاع حصيلة ضحايا الحرب الإسرائيلية على القطا...
2,2024-01-01 00:16:58,AJPalestine,بمشاركة الفنانة الأردنية جولييت عواد.. مسيرة ت...
3,2024-01-01 00:24:58,AJPalestine,عاجل | مراسل #الجزيرة: غارات إسرائيلية على منا...
4,2024-01-01 00:26:11,AJPalestine,من وسط الحرب.. صحفيو غزة يعبرون عن أمنياتهم مع...


In [147]:
# Misbar Data Filter
df = jan_data[jan_data['channel'] == 'MisbarFC']
df.head()

,date,channel,text
66607,2024-01-01 08:39:52,MisbarFC,الفيديو منشور للمرة الأولى يوم 29 ديسمبر/كانون...
66608,2024-01-01 10:34:30,MisbarFC,#زلزال_تركيا_وسوريا، اشتباكات السودان، #زلزال_...
66609,2024-01-01 12:30:19,MisbarFC,🟡المشاهد تعود إلى احتفالات بعيد الميلاد أقيمت ...
66610,2024-01-01 18:05:58,MisbarFC,منذ عام 2016 على الأقل كانت الانتخابات عُرضة ل...
66611,2024-01-01 19:00:00,MisbarFC,ما هي أبرز الأحداث والقضايا التي نُشرت حولها أ...


In [ ]:
# Extract Links from text
def extract_msbr_links(text):
    pattern = r'https://msbr\.co/\S+'
    return re.findall(pattern, str(text))

df["msbr_links"] = df["text"].apply(extract_msbr_links)
df_expanded = df.explode("msbr_links").dropna(subset=["msbr_links"])
df_expanded = df_expanded.rename(columns={"msbr_links": "url", "channel": "platform"})
df_links = df_expanded[["date", "platform", "url"]].reset_index(drop=True)

In [41]:
# Find title and content from link
from newspaper import Article
from tqdm import tqdm  
titles = []
contents = []

for link in tqdm(df_links["url"]):
    try:
        article = Article(link)
        article.download()
        article.parse()
        titles.append(article.title)
        contents.append(article.text)
    except Exception as e:
        titles.append(None)
        contents.append(None)

100%|████████████████████████████████████████████████████████████████████████████████| 206/206 [24:05<00:00,  7.02s/it]


In [150]:
df_links["title"] = titles
df_links["news_content"] = contents
df_links["label"] = None  

df_links["Id"] = range(1, len(df_links) + 1)
final_df = df_links[["Id", "date", "platform",'url', "title", "news_content", "label"]]
final_df.to_csv('final_misbar_v1.csv')
pd.read_excel("C:\\Users\\salma\\OneDrive\\سطح المكتب\\final_misbar_v1.xlsx").head()

,Date,Platform,Title,News Content,Label
0,45292,TikTok,دبابات إسرائيلية في تل أبيب بعد تدميرها على يد...,أكبر مقبرة دبابات في #العالم في #تل_أبيب لدباب...,Fake
1,45292,X,فيديو إضاءة شجرة عيد الميلاد في رام الله,احتفالات رام الله والبيرة برأس السنة 2024\n\nا...,Fake
2,45293,X,فتح باب الهجرة للفلسطينيين الى كندا,عزيزي الفلسطيني وبالتحديد الفزاوي\nكندا فتحت أ...,Fake
3,45293,X,كندا تعلن عن خطة لتهجير الفلسطينيين من غزة وا...,وصلتني معلومات أن الاتفاق قد تم مع حكومتي #كند...,Fake
4,45293,MisbarFC,الجيش الإسرائيلي يفرض رقابة على تغطية الإعلام ...,الأسلحة المستخدمة من قبل قوات الاحتلال الإسرائ...,Real


---
---
## `Government Websites - Manually `

In [151]:
Government_df = pd.read_excel(r"C:\Users\salma\OneDrive\سطح المكتب\NLP3\Government_df.xlsx")
Government_df.head()

,Date,Platform,Title,News Content,Label
0,2024-01-02 00:00:00,وزارة التربية والتعليم العالي,"""التعليم العالي"" تفتح باب التسجيل لترخيص مكاتب...",أعلنت وزارة التعليم العالي والبحث العلمي، اليو...,Real
1,2024-01-04 00:00:00,وزارة التربية والتعليم العالي,"""التعليم العالي"" تعلن عن منح دراسية في الجزائر...",أعلنت وزارة التعليم العالي والبحث العلمي، اليو...,Real
2,2024-01-10 00:00:00,وزارة التربية والتعليم العالي,"""التعليم العالي"" تعلن عن منح دراسية في الصين و...",أعلنت وزارة التعليم العالي والبحث العلمي، اليو...,Real
3,15-1-2024,وزارة التربية والتعليم العالي,"""التعليم العالي"" تدين اقتحام الاحتلال لجامعة ا...",أدانت وزارة التعليم العالي والبحث العلمي اقتحا...,Real
4,2024-01-16 00:00:00,وزارة التربية والتعليم العالي,"""التعليم العالي"" تعلن عن منح دراسية في بنغلادش...",image not exsits\nوحدة العلاقات العامة والدولي...,Real


In [157]:
Government_df['Platform'].value_counts()

الاتحاد الفلسطيني لكرة القدم     15
وزارة التنمية الاجتماعية         10
سلطة جودة البيئة                  9
وزارة التربية والتعليم العالي     8
وزارة الخارجية والمغتربين         5
وزارة الثقافة                     1
وزارة الصحة الفلسطينية            1
Name: Platform, dtype: int64

---
---

## `Concatnate Data Files `
1. Al Jazeera Dataset 
2. Misbar Dataset and Several Platforms Dataset

In [161]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
Al_Jazeera=pd.read_csv("final_jan_1.csv")
misbar=pd.read_excel("C:\\Users\\salma\\OneDrive\\سطح المكتب\\final_misbar_v1.xlsx")
Government_df = pd.read_excel(r"C:\Users\salma\OneDrive\سطح المكتب\NLP3\Government_df.xlsx")

### Check the data type to ensure the concattnate

In [162]:
print(Al_Jazeera.info())
print("_"*50)
print(misbar.info())
print("_"*50)
print(Government_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          240 non-null    object
 1   Platform      240 non-null    object
 2   Title         240 non-null    object
 3   News Content  240 non-null    object
 4   Label         240 non-null    object
dtypes: object(5)
memory usage: 9.5+ KB
None
__________________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          165 non-null    int64 
 1   Platform      165 non-null    object
 2   Title         165 non-null    object
 3   News Content  165 non-null    object
 4   Label         165 non-null    object
dtypes: int64(1), object(4)
memory usage: 6.6+ KB
None
__________________________________________________
<c

In [ ]:
Al_Jazeera['Date'] = pd.to_datetime(Al_Jazeera['Date'], errors='coerce').dt.date
Government_df['Date'] = pd.to_datetime(Government_df['Date'], errors='coerce').dt.date

# Convert Excel serial dates
misbar['Date'] = pd.to_datetime(misbar['Date'], origin='1899-12-30', unit='D').dt.date

In [131]:
real_fake_news = pd.concat([Al_Jazeera, misbar,Government_df], ignore_index=True)
real_fake_news

,Date,Platform,Title,News Content,Label
0,2024-01-01,Aljazeera,عشية رأس السنة.. مظاهرات داعمة لغزة حول العالم,احتفل العالم بالسنة الميلادية الجديدة 2024، ول...,Real
1,2024-01-01,Aljazeera,"كيف تفاعل جمهور المنصات مع قصة ""الجندي الإسرائ...",لم يستغرب جمهور وسائل التواصل الاجتماعي من الف...,Real
2,2024-01-01,Aljazeera,ما وراء سحب واشنطن حاملة طائرات رغم استمرار حر...,أجمع خبراء على أن قرار واشنطن سحب حاملة الطائر...,Real
3,2024-01-01,Aljazeera,مسؤول أميركي: سحب إسرائيل بعض قواتها من غزة بد...,قال مسؤول أميركي اليوم الاثنين إن قرار إسرائيل...,Real
4,2024-01-10,Aljazeera,أبرز تطورات اليوم الـ96 من الحرب الإسرائيلية ع...,في اليوم الـ96 من العدوان الإسرائيلي على غزة، ...,Real
...,...,...,...,...,...
449,2024-01-11,وزارة الصحة الفلسطينية,تعلن وزارة الصحة عن بدء التسجيل لامتحان مزاولة...,علن وزارة الصحة الفلسطينية وبالتنسيق مع نقابة ...,Real
450,2024-01-18,وزارة التنمية الاجتماعية,د. مجدلاني يقلد المدير القُطري لبرنامج الأغذية...,باسم رئيس دولة فلسطين محمود عباس، قلد وزير الت...,Real
451,2024-01-30,وزارة التنمية الاجتماعية,تصريح صحفي هام من وزارة التنمية الاجتماعية\n,تؤكد وزارة التنمية الاجتماعية على استمرار عمل ...,Real
452,2024-01-25,وزارة التنمية الاجتماعية,التنمية الاجتماعية والمجلس النرويجي للاجئين يو...,وقع وزير التنمية الاجتماعية د.أحمد مجدلاني، في...,Real


### `Filtring Dataset to remain the news just related to Palestine`

In [132]:
# Define Arabic keywords related to Palestine
palestine_keywords = ['العالي','القصف','الفدائي','الحرب','فلسطين', 'غزة', 'القدس', 'الضفة', 'الاحتلال', 'إسرائيلي', 'شهيد', 'قصف','نابلس','الداخل','المجازر','فلسطينية','حماس','المحتل','إسرائيل','بايدن','المقاومة']
combined_text = real_fake_news["Title"].fillna('') + " " + real_fake_news["News Content"].fillna('')
mask = combined_text.apply(lambda x: any(keyword in x for keyword in palestine_keywords))

# Filter the DataFrame
real_fake_news = real_fake_news[mask==True]
real_fake_news

,Date,Platform,Title,News Content,Label
0,2024-01-01,Aljazeera,عشية رأس السنة.. مظاهرات داعمة لغزة حول العالم,احتفل العالم بالسنة الميلادية الجديدة 2024، ول...,Real
1,2024-01-01,Aljazeera,"كيف تفاعل جمهور المنصات مع قصة ""الجندي الإسرائ...",لم يستغرب جمهور وسائل التواصل الاجتماعي من الف...,Real
2,2024-01-01,Aljazeera,ما وراء سحب واشنطن حاملة طائرات رغم استمرار حر...,أجمع خبراء على أن قرار واشنطن سحب حاملة الطائر...,Real
3,2024-01-01,Aljazeera,مسؤول أميركي: سحب إسرائيل بعض قواتها من غزة بد...,قال مسؤول أميركي اليوم الاثنين إن قرار إسرائيل...,Real
4,2024-01-10,Aljazeera,أبرز تطورات اليوم الـ96 من الحرب الإسرائيلية ع...,في اليوم الـ96 من العدوان الإسرائيلي على غزة، ...,Real
...,...,...,...,...,...
448,2024-01-13,وزارة الخارجية والمغتربين\n,السفيرة الخالدي تطلع وزير خارجية سلوفاكيا على ...,أطلعت سفيرة دولة فلسطين لدى سلوفاكيا صفاء الخا...,Real
449,2024-01-11,وزارة الصحة الفلسطينية,تعلن وزارة الصحة عن بدء التسجيل لامتحان مزاولة...,علن وزارة الصحة الفلسطينية وبالتنسيق مع نقابة ...,Real
450,2024-01-18,وزارة التنمية الاجتماعية,د. مجدلاني يقلد المدير القُطري لبرنامج الأغذية...,باسم رئيس دولة فلسطين محمود عباس، قلد وزير الت...,Real
452,2024-01-25,وزارة التنمية الاجتماعية,التنمية الاجتماعية والمجلس النرويجي للاجئين يو...,وقع وزير التنمية الاجتماعية د.أحمد مجدلاني، في...,Real


In [133]:
real_fake_news['Label'].value_counts()

Real    347
Fake    100
Name: Label, dtype: int64

### `Simple Cleaning to Data remove :`
- URLs
- Symbols  
- References 
- links 


In [ ]:
import re

def clean_arabic_text(text):
    if pd.isna(text):
        return ""
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        if (re.search(r"pic\.twitter\.com.*@.*\(\@.*\).*\d{4}", line) or
            re.search(r"pic\.twitter\.com.*\d{4}", line) or
            re.fullmatch(r"\s*(إعلان|مبروووك)\s*", line) or
            re.search(r"@\w+.*\d{4}", line)):
            continue
        cleaned_lines.append(line)
    
    text = ' '.join(cleaned_lines)  # join lines with space to remove \n
    
    # Remove URLs, mentions, hashtags, emojis, Latin chars, specific words, and extra spaces
    text = re.sub(r"http\S+|pic\.twitter\.com\S+|@\w+|#\w+", "", text)
    text = re.sub(r"[^\w\s\u0600-\u06FF]", " ", text)  # keep Arabic letters and spaces
    text = re.sub(r"[A-Za-z؟!]", " ", text)
    text = re.sub(r"الحرب على غزة مباشر..", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Apply to columns
real_fake_news["Title"] = real_fake_news["Title"].apply(clean_arabic_text)
real_fake_news["News Content"] = real_fake_news["News Content"].apply(clean_arabic_text)


### Add `Id` column as a series number

In [ ]:
# Add ID column
real_fake_news['ID'] = [i+1 for i in range(len(real_fake_news))]

# Move 'ID' column to the front (index 0)
cols = ['ID'] + [col for col in real_fake_news.columns if col != 'ID']
real_fake_news = real_fake_news[cols]
real_fake_news

In [143]:
real_fake_news['Platform'].value_counts()

Aljazeera                        240
X                                 64
الاتحاد الفلسطيني لكرة القدم      30
وزارة التنمية الاجتماعية          18
Facebook                          17
سلطة جودة البيئة                  16
MisbarFC                          16
وزارة التربية والتعليم العالي     16
وزارة الخارجية والمغتربين         10
YouTube                            6
Instagram                          5
وزارة الثقافة                      2
وزارة الصحة الفلسطينية             2
TikTok                             2
tayqan                             1
Twitter                            1
chayyek                            1
Name: Platform, dtype: int64

### Save the dataset  as `GroupC_NLP_Task3.csv` file

In [136]:
real_fake_news.to_csv("GroupC_NLP_Task3.csv", index=False, encoding='utf-8-sig')

## ===================*`Thank you !`*=====================